In [ ]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
import pandas as pd
df_sampled_unscaled = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_unscaled.csv")
df_sampled_scaled = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_scaled.csv")

In [ ]:
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

# **Random Forest baseline class imbalance**

+ weighted f1 score as the scoring metric (consistent with XGB)

In [ ]:
import numpy as np
import pandas as pd
import joblib
from google.colab import drive
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Load df Google Colab
drive.mount('/content/drive')
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

df_final = df_final.astype(np.int32)

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import joblib

In [ ]:
# Define feature and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [100, 200, 250, 300],
    'max_depth': [20, 30, 40, 50],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric
f1_weighted_scorer = make_scorer(f1_score, average='weighted')


In [ ]:

# Initialize the Randomized Search with class_weight='balanced'
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(class_weight='balanced', random_state=42),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring=f1_weighted_scorer)

# Define output lists
outer_scores = []
val_scores = []
best_params_list = []

# Perform nested CV
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Store validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (f1_weighted)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested CV
print(f"Nested Cross-Validation F1-score: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train the best model with class_weight='balanced'
best_model = RandomForestClassifier(**best_params, class_weight='balanced', random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test set performance for best performing model:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

# Save the best performing model
joblib_file = "/content/drive/My Drive/Thesis/Models/Baseline_RF_Class_imb.pkl"
joblib.dump(best_model, joblib_file)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Nested Cross-Validation F1-score: 0.40 ± 0.00
Validation Set Performance:
Accuracy: 0.41 ± 0.00
Precision: 0.40 ± 0.00
Recall: 0.41 ± 0.00
F1 Score: 0.40 ± 0.00
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 30}
Fold 2: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 30}
Fold 3: {'n_estimators': 250, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 40}
Fold 4: {'n_estimators': 250, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 50}
Fold 5: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 30}
Test set performance for best performing model:
Accuracy: 0.41
Precision: 0.40
Recall: 0.41
F1 Score: 0.41
Parameters for the best performing model:
{'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 30}


['/content/drive/My Drive/Thesis/Models/Baseline_RF_Class_imb.pkl']

# **XGBoost baseline, met training output**

In [11]:
pip install xgboost

In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import KFold, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

In [27]:
# Load in the data
drive.mount('/content/drive')
df_final = pd.read_csv('/content/drive/My Drive/Thesis/Data/df_final_2.csv')

df_final = df_final.astype(np.int32)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the "Rating" column
df_final['Rating'] = label_encoder.fit_transform(df_final['Rating'])

In [29]:
# Define feature and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

# Define the parameter grid for tuning
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric
f1_weighted_scorer = make_scorer(f1_score, average='weighted')

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import joblib
from xgboost import XGBClassifier

def evaluate_model(model, X, y, dataset_name=""):
    predictions = model.predict(X)
    acc = accuracy_score(y, predictions)
    prec = precision_score(y, predictions, average='weighted')
    rec = recall_score(y, predictions, average='weighted')
    f1 = f1_score(y, predictions, average='weighted')

    return acc, prec, rec, f1

def evaluate_model_print(model, X, y, dataset_name=""):
    predictions = model.predict(X)
    acc = accuracy_score(y, predictions)
    prec = precision_score(y, predictions, average='weighted')
    rec = recall_score(y, predictions, average='weighted')
    f1 = f1_score(y, predictions, average='weighted')

    print(f"{dataset_name} Set Performance:")
    print(f"Accuracy: {acc:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"Recall: {rec:.2f}")
    print(f"F1 Score: {f1:.2f}")

    return acc, prec, rec, f1


def print_average_performance(scores, dataset_name=""):
    scores = np.array(scores)
    print(f"{dataset_name} Set Performance (Averaged):")
    print(f"Accuracy: {scores[:, 0].mean():.2f} ± {scores[:, 0].std():.2f}")
    print(f"Precision: {scores[:, 1].mean():.2f} ± {scores[:, 1].std():.2f}")
    print(f"Recall: {scores[:, 2].mean():.2f} ± {scores[:, 2].std():.2f}")
    print(f"F1 Score: {scores[:, 3].mean():.2f} ± {scores[:, 3].std():.2f}")

def print_hyperparameter_performance(cv_results):
    print("Average F1 scores for all hyperparameters tried:")
    for params, mean_score in zip(cv_results['params'], cv_results['mean_test_score']):
        print(f"Parameters: {params}, Average F1 Score: {mean_score:.2f}")

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=XGBClassifier(tree_method='hist', device='cuda', eval_metric='merror'),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring=f1_weighted_scorer,
    n_jobs=-1
)

# Define output lists
outer_scores = []
val_scores = []
train_scores = []
best_params_list = []

# Perform nested CV on the training data
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the training set
    train_scores.append(evaluate_model(best_model, X_train_fold, y_train_fold, "Training"))

    # Evaluate on the validation set
    val_scores.append(evaluate_model(best_model, X_val_fold, y_val_fold, "Validation"))

    # Store the outer fold score (f1_weighted)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested CV
print(f"Nested Cross-Validation F1 Score: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")
print("")
print("Average metrics for all folds:")
# Print averaged training set performance for each fold
print_average_performance(train_scores, "Training")
# Print averaged validation set performance for each fold
print_average_performance(val_scores, "Validation")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Print average F1 scores for all hyperparameters tried
print_hyperparameter_performance(random_search.cv_results_)

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train best model on the entire training data
best_model = XGBClassifier(**best_params, tree_method='hist', device='cuda', eval_metric='merror', random_state=42)
best_model.fit(X_train, y_train)

print("Evaluation of best performing model:")
# Evaluate on the training set
evaluate_model_print(best_model, X_train, y_train, "Training")
# Evaluate on the test set
evaluate_model_print(best_model, X_test, y_test, "Test")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

joblib.dump(best_model, "/content/drive/My Drive/Thesis/Models/Baseline_XGB_training_info.pkl")

Nested Cross-Validation F1 Score: 0.41 ± 0.01

Average metrics for all folds:
Training Set Performance (Averaged):
Accuracy: 0.56 ± 0.05
Precision: 0.58 ± 0.05
Recall: 0.56 ± 0.05
F1 Score: 0.55 ± 0.06
Validation Set Performance (Averaged):
Accuracy: 0.43 ± 0.01
Precision: 0.43 ± 0.00
Recall: 0.43 ± 0.01
F1 Score: 0.41 ± 0.01
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}
Fold 2: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.2}
Fold 3: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.2}
Fold 4: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.2}
Fold 5: {'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.2}
Average F1 scores for all hyperparameters tried:
Parameters: {'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.1}, Average F1 Score: 0.38
Parameters: {'n_estimators': 400, 'max_depth': 3, 'learning_rate': 0.1}, Average F1 Score: 0.36
Parameters: {'n_estimators': 200, 'max_

['/content/drive/My Drive/Thesis/Models/Baseline_XGB_training_info.pkl']

Too much information

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import joblib
from xgboost import XGBClassifier

def evaluate_model(model, X, y, dataset_name=""):
    predictions = model.predict(X)
    acc = accuracy_score(y, predictions)
    prec = precision_score(y, predictions, average='weighted')
    rec = recall_score(y, predictions, average='weighted')
    f1 = f1_score(y, predictions, average='weighted')

    print(f"{dataset_name} Set Performance:")
    print(f"Accuracy: {acc:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"Recall: {rec:.2f}")
    print(f"F1 Score: {f1:.2f}")

    return acc, prec, rec, f1

def print_average_performance(scores, dataset_name=""):
    scores = np.array(scores)
    print(f"{dataset_name} Set Performance (Averaged):")
    print(f"Accuracy: {scores[:, 0].mean():.2f} ± {scores[:, 0].std():.2f}")
    print(f"Precision: {scores[:, 1].mean():.2f} ± {scores[:, 1].std():.2f}")
    print(f"Recall: {scores[:, 2].mean():.2f} ± {scores[:, 2].std():.2f}")
    print(f"F1 Score: {scores[:, 3].mean():.2f} ± {scores[:, 3].std():.2f}")

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=XGBClassifier(tree_method='hist', device='cuda', eval_metric='merror'),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring=f1_weighted_scorer,
    n_jobs=-1
)

# Define output lists
outer_scores = []
val_scores = []
train_scores = []
best_params_list = []
all_param_scores = []

# Perform nested CV on the training data
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the training set
    train_scores.append(evaluate_model(best_model, X_train_fold, y_train_fold, "Training"))

    # Evaluate on the validation set
    val_scores.append(evaluate_model(best_model, X_val_fold, y_val_fold, "Validation"))

    # Store the outer fold score (f1_weighted)
    outer_scores.append(random_search.best_score_)

    # Store all parameter configurations and their scores
    all_param_scores.append(random_search.cv_results_['mean_test_score'])

# Print the performance of the nested CV
print(f"Nested Cross-Validation F1 Score: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print averaged training set performance for each fold
print_average_performance(train_scores, "Training")

# Print averaged validation set performance for each fold
print_average_performance(val_scores, "Validation")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Print F1 scores of different parameter configurations tried during hyperparameter tuning
print("F1 scores of different parameter configurations tried during hyperparameter tuning:")
for i, param_scores in enumerate(all_param_scores):
    print(f"Fold {i+1}: {param_scores}")

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train best model on the entire training data
best_model = XGBClassifier(**best_params, tree_method='hist', device='cuda', eval_metric='merror', random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on the training set
evaluate_model(best_model, X_train, y_train, "Training")

# Evaluate on the test set
evaluate_model(best_model, X_test, y_test, "Test")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

joblib.dump(best_model, "/content/drive/My Drive/Thesis/Models/Baseline_XGB_training_info.pkl")

Training Set Performance:
Accuracy: 0.59
Precision: 0.62
Recall: 0.59
F1 Score: 0.58
Validation Set Performance:
Accuracy: 0.44
Precision: 0.44
Recall: 0.44
F1 Score: 0.42
Training Set Performance:
Accuracy: 0.63
Precision: 0.65
Recall: 0.63
F1 Score: 0.62
Validation Set Performance:
Accuracy: 0.44
Precision: 0.44
Recall: 0.44
F1 Score: 0.42
Training Set Performance:
Accuracy: 0.59
Precision: 0.62
Recall: 0.59
F1 Score: 0.58
Validation Set Performance:
Accuracy: 0.44
Precision: 0.43
Recall: 0.44
F1 Score: 0.42


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Training Set Performance:
Accuracy: 0.63
Precision: 0.65
Recall: 0.63
F1 Score: 0.62
Validation Set Performance:
Accuracy: 0.44
Precision: 0.43
Recall: 0.44
F1 Score: 0.42
Training Set Performance:
Accuracy: 0.59
Precision: 0.61
Recall: 0.59
F1 Score: 0.58
Validation Set Performance:
Accuracy: 0.43
Precision: 0.43
Recall: 0.43
F1 Score: 0.42
Nested Cross-Validation F1 Score: 0.41 ± 0.00
Training Set Performance (Averaged):
Accuracy: 0.61 ± 0.02
Precision: 0.63 ± 0.02
Recall: 0.61 ± 0.02
F1 Score: 0.60 ± 0.02
Validation Set Performance (Averaged):
Accuracy: 0.44 ± 0.00
Precision: 0.43 ± 0.00
Recall: 0.44 ± 0.00
F1 Score: 0.42 ± 0.00
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.2}
Fold 2: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}
Fold 3: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.2}
Fold 4: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}
Fold 5: {'n_estimators': 300, 'max_d

['/content/drive/My Drive/Thesis/Models/Baseline_XGB_training_info.pkl']

# OLD CODE! NOT USED IN THESIS

## 29/10 Random Forest with hyperparameter tuning (new sample_df)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

In [ ]:
# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)

# Perform nested cross-validation
outer_scores = []
val_scores = []

for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Append validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Fit the best model found on the entire training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Nested Cross-Validation Accuracy: 0.41 ± 0.00
Validation Set Performance:
Accuracy: 0.41 ± 0.00
Precision: 0.42 ± 0.00
Recall: 0.41 ± 0.00
F1 Score: 0.38 ± 0.00
Test Set Performance:
Accuracy: 0.42
Precision: 0.43
Recall: 0.42
F1 Score: 0.39


## 30/10 Random Forest with hyperparameter tuning, sample_df_3010



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/sample_df_3010.csv")

In [ ]:
# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

In [ ]:
# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

In [ ]:
# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)



# Perform nested cross-validation
outer_scores = []
val_scores = []

for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Append validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Fit the best model found on the entire training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Nested Cross-Validation Accuracy: 0.40 ± 0.00
Validation Set Performance:
Accuracy: 0.41 ± 0.00
Precision: 0.42 ± 0.00
Recall: 0.41 ± 0.00
F1 Score: 0.38 ± 0.00
Test Set Performance:
Accuracy: 0.41
Precision: 0.42
Recall: 0.41
F1 Score: 0.38


## Gradient boosting with hyperparameter tuning (-2 features) -> Accuracy 0.46


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [ ]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
df_sampled_unscaled = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_unscaled.csv")
df_sampled_scaled = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_sampled_scaled.csv")

In [ ]:
df_final = df_sampled_unscaled.copy()
df_final = df_final.drop(columns=['Dev_movie_avg',	'Total_ratings_per_user'])

In [ ]:
# Separate features and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

In [ ]:
# Split data into 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
# Define evaluation function
def evaluate_performance(y_true, y_pred):
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='macro'):.2f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='macro'):.2f}")
    print(f"F1 Score: {f1_score(y_true, y_pred, average='macro'):.2f}")

In [ ]:
from joblib import Parallel, delayed

# Define the parameter grid for tuning
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 7, 10, 20]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1  # Utilize all available cores for parallel processing
)

# Perform nested cross-validation and store best parameters for each fold
nested_scores = []
best_params = []
best_models = []

def fit_and_evaluate(train_idx, test_idx):
    random_search.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
    best_model = random_search.best_estimator_
    return random_search.best_score_, random_search.best_params_, best_model

results = Parallel(n_jobs=-1)(delayed(fit_and_evaluate)(train_idx, test_idx) for train_idx, test_idx in outer_cv.split(X_train, y_train))

for score, params, model in results:
    nested_scores.append(score)
    best_params.append(params)
    best_models.append(model)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(nested_scores):.2f} ± {np.std(nested_scores):.2f}")

# Print the best parameters found for each fold
print("Best parameters for each fold:")
for params in best_params:
    print(params)

# Use the best model found in the outer loop
final_best_model = best_models[-1]

# Evaluate the best model on the test set
test_predictions = final_best_model.predict(X_test)
print("Test Set Performance:")
evaluate_performance(y_test, test_predictions)


Nested Cross-Validation Accuracy: 0.46 ± 0.00
Best parameters for each fold:
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.1}
Test Set Performance:
Accuracy: 0.46
Precision: 0.46
Recall: 0.38
F1 Score: 0.40


## **BASELINE RANDOM FOREST 10/11, df_final_2**

> Good dataframe, not good that gridcv is ran twice!!



In [ ]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

pd.set_option('display.max_columns', None)
df_final.head()

,UserID,MovieID,Rating,Age,Year,Month,Day,Hour,Release_year,Time_release_to_rating,Total_ratings_per_movie,Total_ratings_per_user,Female,Male,Academic/educator,Artist,Clerical/admin,College/grad student,Customer service,Doctor/health care,Executive/managerial,Farmer,Homemaker,K-12 student,Lawyer,Other or not specified,Programmer,Retired,Sales/marketing,Scientist,Self-employed,Technician/engineer,Tradesman/craftsman,Unemployed,Writer,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Favourite_Action,Favourite_Adventure,Favourite_Animation,Favourite_Children's,Favourite_Comedy,Favourite_Crime,Favourite_Documentary,Favourite_Drama,Favourite_Fantasy,Favourite_Film-Noir,Favourite_Horror,Favourite_Musical,Favourite_Mystery,Favourite_Romance,Favourite_Sci-Fi,Favourite_Thriller,Favourite_War,Favourite_Western
0,5616,3590,3,4,2000,5,24,2,1974,26,85,130,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,4060,21,4,2,2000,8,5,15,1995,5,1356,256,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,1125,3273,2,1,2000,11,22,16,2000,0,577,580,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,3410,585,4,3,2000,8,27,22,1995,5,418,731,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,3675,1374,4,3,2000,8,15,18,1982,18,1448,849,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [100, 200, 250, 300],
    'max_depth': [20, 30, 40, 50],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Turning booleans into numerical values
#df_final = np.nan_to_num(df_final)
df_final.astype(np.int32)

array([[5616, 3590,    3, ...,    0,    0,    1],
       [4060,   21,    4, ...,    0,    0,    1],
       [1125, 3273,    2, ...,    0,    0,    0],
       ...,
       [1100, 3525,    1, ...,    0,    0,    0],
       [1971, 2355,    4, ...,    0,    1,    0],
       [5136, 1429,    4, ...,    0,    0,    0]], dtype=int32)

In [ ]:
# Initialize the Ranodom Search
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy',
                                   random_state=42)

# Define output lists
outer_scores = []
val_scores = []
best_params_list = []

# Perform nested cross-validation
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Append validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested cross-validation
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Fit the best model found on the entire training set
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the best parameters of the best-performing model
print("Best Parameters of the Best-Performing Model:")
print(random_search.best_params_)

Nested Cross-Validation Accuracy: 0.41 ± 0.00
Validation Set Performance:
Accuracy: 0.42 ± 0.00
Precision: 0.42 ± 0.00
Recall: 0.42 ± 0.00
F1 Score: 0.39 ± 0.00
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}
Fold 2: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 30}
Fold 3: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}
Fold 4: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 30}
Fold 5: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 30}
Test Set Performance:
Accuracy: 0.42
Precision: 0.43
Recall: 0.42
F1 Score: 0.40
Best Parameters of the Best-Performing Model:
{'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 40}


## Collaborative Filtering, matrix

In [ ]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class ImputationMatrixFactorization:
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def impute_missing_values(self):
        imputer = SimpleImputer(strategy='mean')
        self.R_imputed = imputer.fit_transform(self.R)

    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R_imputed[np.where(self.R_imputed != 0)])

        self.samples = [
            (i, j, self.R_imputed[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            if (i+1) % 10 == 0:
                print(f"Iteration: {i+1}; error = {mse:.4f}")

    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += (self.R[x, y] - predicted[x, y]) ** 2
        return np.sqrt(error)

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)



In [ ]:
R_df = df_final.pivot(index='UserID', columns='MovieID', values='Rating')
R = R_df.values

imf = ImputationMatrixFactorization(R, K=2, alpha=0.01, beta=0.01, iterations=100)
imf.impute_missing_values()
imf.train()
predicted_matrix = imf.full_matrix()
print(predicted_matrix)

# Evaluation
def get_true_pred_ratings(df, predicted_df):
    true_ratings = []
    predicted_ratings = []
    for row in df.itertuples():
        user_id = row.UserID
        movie_id = row.MovieID
        true_ratings.append(row.Rating)
        predicted_ratings.append(predicted_df.loc[user_id, movie_id])
    return np.array(true_ratings), np.array(predicted_ratings)

true_ratings, predicted_ratings = get_true_pred_ratings(df_final, pd.DataFrame(predicted_matrix, columns=R_df.columns, index=R_df.index))

precision = precision_score(true_ratings, predicted_ratings.round(), average='weighted', zero_division=0)
recall = recall_score(true_ratings, predicted_ratings.round(), average='weighted', zero_division=0)
f1 = f1_score(true_ratings, predicted_ratings.round(), average='weighted', zero_division=0)
accuracy = accuracy_score(true_ratings, predicted_ratings.round())

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Iteration: 10; error = nan
Iteration: 20; error = nan
Iteration: 30; error = nan
Iteration: 40; error = nan
Iteration: 50; error = nan
Iteration: 60; error = nan
Iteration: 70; error = nan
Iteration: 80; error = nan
Iteration: 90; error = nan
Iteration: 100; error = nan
[[4.46010186 3.60829331 3.43042852 ... 4.32580485 2.65673402 3.87182357]
 [4.14812481 3.34335054 3.14951091 ... 3.96385389 2.55786762 3.69507351]
 [4.27883658 3.44457187 3.23395441 ... 3.92351679 3.35727072 4.12188705]
 ...
 [4.46996054 4.18968623 3.91941883 ... 4.49668633 2.21048452 4.07896634]
 [4.19810411 3.50903944 3.27162849 ... 3.85862852 3.26002517 4.13844656]
 [3.44117644 1.93717885 1.8060465  ... 2.67968091 3.91627503 3.46236176]]
Precision: 0.5177
Recall: 0.4668
F1 Score: 0.4472
Accuracy: 0.4668


## Random Forest old baseline, no class imbalance

In [ ]:
import numpy as np
import pandas as pd
import joblib
from google.colab import drive
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Load df Google Colab
drive.mount('/content/drive')
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

df_final = df_final.astype(np.int32)

Mounted at /content/drive


In [ ]:
# Define feature and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [100, 200, 250, 300],
    'max_depth': [20, 30, 40, 50],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
# Initialize the Randomized Search
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist, n_iter=15,
                                   cv=inner_cv, scoring='accuracy')

# Define output lists
outer_scores = []
val_scores = []
best_params_list = []

# Perform nested CV
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Store validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (accuracy)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested CV
print(f"Nested Cross-Validation Accuracy: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train the best model
best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test set performance for best performing model:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

# Save the best performing model
joblib_file = "/content/drive/My Drive/Thesis/Models/Baseline_RF.pkl"
joblib.dump(best_model, joblib_file)


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Nested Cross-Validation Accuracy: 0.41 ± 0.00
Validation Set Performance:
Accuracy: 0.42 ± 0.00
Precision: 0.42 ± 0.00
Recall: 0.42 ± 0.00
F1 Score: 0.39 ± 0.00
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 30}
Fold 2: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 40}
Fold 3: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 40}
Fold 4: {'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 40}
Fold 5: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 40}
Test set performance for best performing model:
Accuracy: 0.42
Precision: 0.43
Recall: 0.42
F1 Score: 0.40
Parameters for the best performing model:
{'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 40}


['/content/drive/My Drive/Thesis/Models/Baseline_RF.pkl']

## XGB with compute_sample_weight




In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import KFold, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

In [ ]:
# Load in the data
drive.mount('/content/drive')
df_final = pd.read_csv('/content/drive/My Drive/Thesis/Data/df_final_2.csv')

df_final = df_final.astype(np.int32)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the "Rating" column
df_final['Rating'] = label_encoder.fit_transform(df_final['Rating'])

In [ ]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, accuracy_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import XGBClassifier
import numpy as np
import joblib

# Define feature and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Compute sample weights
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# Define the parameter grid for tuning
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric
f1_weighted_scorer = make_scorer(f1_score, average='weighted')

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=XGBClassifier(tree_method='hist', device='cuda', eval_metric='merror'),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring=f1_weighted_scorer,
    n_jobs=-1
)

# Define output lists
outer_scores = []
val_scores = []
best_params_list = []

# Perform nested CV on the training data
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    sample_weights_fold = sample_weights[train_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold, sample_weight=sample_weights_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Store validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (f1_weighted)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested CV
print(f"Nested Cross-Validation F1 Score: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train best model on the entire training data with sample weights
best_model = XGBClassifier(**best_params, tree_method='hist', device='cuda', eval_metric='merror', random_state=42)
best_model.fit(X_train, y_train, sample_weight=compute_sample_weight(class_weight='balanced', y=y_train))

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test set performance for best performing model:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

# Save the best performing model
joblib_file = "/content/drive/My Drive/Thesis/Models/Baseline_XGB_class_imbalance.pkl"
joblib.dump(best_model, joblib_file)

Nested Cross-Validation F1 Score: 0.38 ± 0.00
Validation Set Performance:
Accuracy: 0.38 ± 0.00
Precision: 0.41 ± 0.00
Recall: 0.38 ± 0.00
F1 Score: 0.38 ± 0.01
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.2}
Fold 2: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.2}
Fold 3: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.2}
Fold 4: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.2}
Fold 5: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.2}
Test set performance for best performing model:
Accuracy: 0.39
Precision: 0.42
Recall: 0.39
F1 Score: 0.39
Parameters for the best performing model:
{'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.2}


['/content/drive/My Drive/Thesis/Models/Baseline_XGB_class_imbalance.pkl']

## SVD

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/df_final_2.csv")

Mounted at /content/drive


In [ ]:
# Sample 2.5% of the dataframe
df = df_final.sample(frac=0.005, random_state=42)

In [ ]:
df.head()

,UserID,MovieID,Rating,Age,Year,Month,Day,Hour,Release_year,Time_release_to_rating,...,Favourite_Fantasy,Favourite_Film-Noir,Favourite_Horror,Favourite_Musical,Favourite_Mystery,Favourite_Romance,Favourite_Sci-Fi,Favourite_Thriller,Favourite_War,Favourite_Western
28385,824,1225,5,3,2000,11,28,3,1984,16,...,False,False,False,False,False,False,False,False,True,False
282334,1943,2879,3,1,2000,11,20,3,1990,10,...,False,False,False,False,False,False,False,False,False,False
312857,4605,3129,3,2,2000,7,21,18,1999,1,...,False,False,False,False,False,False,False,False,False,True
317108,3199,1210,4,1,2000,9,11,0,1983,17,...,False,False,False,False,False,False,False,False,True,False
321785,3367,3091,4,5,2000,9,27,18,1980,20,...,False,True,False,False,False,False,False,False,False,False


In [ ]:
# Assuming df is your dataframe with columns: UserID, MovieID, Rating
n_users = df['UserID'].nunique()
n_items = df['MovieID'].nunique()

In [ ]:
# Create a user-item matrix
user_item_matrix = df.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0).to_numpy()


In [ ]:
# Initialize parameters
n_factors = 50  # Number of latent factors
learning_rate = 0.01
regularization = 0.1
n_epochs = 20

# Initialize user and item matrices
U = np.random.normal(scale=1./n_factors, size=(n_users, n_factors))
V = np.random.normal(scale=1./n_factors, size=(n_items, n_factors))

# Indicator matrix for existing ratings
indicator = (user_item_matrix > 0).astype(int)

In [ ]:
# Training with SGD
for epoch in range(n_epochs):
    for i in range(n_users):
        for j in range(n_items):
            if indicator[i, j] > 0:
                # Compute the prediction error
                prediction = np.dot(U[i, :], V[j, :])
                error = user_item_matrix[i, j] - prediction

                # Update user and item latent factors
                U[i, :] += learning_rate * (error * V[j, :] - regularization * U[i, :])
                V[j, :] += learning_rate * (error * U[i, :] - regularization * V[j, :])

    # Compute the total loss (optional)
    total_loss = np.sum((indicator * (user_item_matrix - np.dot(U, V.T)))**2) + regularization * (np.sum(U**2) + np.sum(V**2))
    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {total_loss}')

# Predict the missing ratings
predicted_ratings = np.dot(U, V.T)

# Convert the predicted ratings to a DataFrame
predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=df['MovieID'].unique(), index=df['UserID'].unique())


Epoch 1/20, Loss: 2910.2751329225407
Epoch 2/20, Loss: 2907.877243140558
Epoch 3/20, Loss: 2905.4452604807243
Epoch 4/20, Loss: 2902.9555676678538
Epoch 5/20, Loss: 2900.3839332504967
Epoch 6/20, Loss: 2897.7052123159483
Epoch 7/20, Loss: 2894.8930389775082
Epoch 8/20, Loss: 2891.919507061959
Epoch 9/20, Loss: 2888.754835794117
Epoch 10/20, Loss: 2885.367017752993
Epoch 11/20, Loss: 2881.7214470309086
Epoch 12/20, Loss: 2877.7805264523954
Epoch 13/20, Loss: 2873.503254017359
Epoch 14/20, Loss: 2868.844790564453
Epoch 15/20, Loss: 2863.7560131789264
Epoch 16/20, Loss: 2858.183062296759
Epoch 17/20, Loss: 2852.0668950077256
Epoch 18/20, Loss: 2845.342862957065
Epoch 19/20, Loss: 2837.9403406684273
Epoch 20/20, Loss: 2829.7824391211293


In [ ]:
# Example: Recommend 10 movies for user with UserID 1
user_id = 824
user_ratings = predicted_ratings_df.loc[user_id].sort_values(ascending=False)
print(user_ratings.head(10))

2694    0.025831
2193    0.016504
1266    0.016426
1196    0.014798
3404    0.012192
1810    0.011734
260     0.011194
923     0.010700
3481    0.010421
2066    0.009654
Name: 824, dtype: float64


In [ ]:
# Function to calculate RMSE
def calculate_rmse(actual, predicted, indicator):
    # Only consider the actual ratings (where indicator is 1)
    error = indicator * (actual - predicted)
    mse = np.sum(error**2) / np.sum(indicator)
    rmse = np.sqrt(mse)
    return rmse

# Actual ratings matrix (user-item matrix)
actual_ratings = user_item_matrix

# Predicted ratings matrix
predicted_ratings = np.dot(U, V.T)

# Indicator matrix for existing ratings
indicator = (actual_ratings > 0).astype(int)

# Calculate RMSE
rmse = calculate_rmse(actual_ratings, predicted_ratings, indicator)
print(f'RMSE: {rmse}')

RMSE: 3.7601852611228685


## Hyperparameter optimized SVD

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Load the dataset
# Assuming df is your dataframe with columns: UserID, MovieID, Rating
n_users = df['UserID'].nunique()
n_items = df['MovieID'].nunique()

# Create a user-item matrix
user_item_matrix = df.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0).to_numpy()

# Hyperparameters to optimize
n_factors_list = [10, 20, 50]
learning_rate_list = [0.001, 0.01, 0.1]
regularization_list = [0.01, 0.1, 1]

# K-Fold Cross Validation
outer_kf = KFold(n_splits=5, shuffle=True, random_state=42)
inner_kf = KFold(n_splits=3, shuffle=True, random_state=42)

best_params = None
best_rmse = float('inf')

for train_index, test_index in outer_kf.split(user_item_matrix):
    train_data = user_item_matrix[train_index]
    test_data = user_item_matrix[test_index]

    for n_factors in n_factors_list:
        for learning_rate in learning_rate_list:
            for regularization in regularization_list:
                inner_rmse = []

                for inner_train_index, inner_val_index in inner_kf.split(train_data):
                    inner_train_data = train_data[inner_train_index]
                    inner_val_data = train_data[inner_val_index]

                    # Initialize user and item matrices
                    U = np.random.normal(scale=1./n_factors, size=(n_users, n_factors))
                    V = np.random.normal(scale=1./n_factors, size=(n_items, n_factors))

                    # Indicator matrix for existing ratings
                    indicator_train = (inner_train_data > 0).astype(int)
                    indicator_val = (inner_val_data > 0).astype(int)

                    # Training with SGD
                    for epoch in range(20):
                        for i in range(n_users):
                            for j in range(n_items):
                                if indicator_train[i, j] > 0:
                                    # Compute the prediction error
                                    prediction = np.dot(U[i, :], V[j, :])
                                    error = inner_train_data[i, j] - prediction

                                    # Update user and item latent factors
                                    U[i, :] += learning_rate * (error * V[j, :] - regularization * U[i, :])
                                    V[j, :] += learning_rate * (error * U[i, :] - regularization * V[j, :])

                    # Predict the validation ratings
                    predicted_val_ratings = np.dot(U, V.T)

                    # Calculate RMSE for validation set
                    val_rmse = np.sqrt(mean_squared_error(inner_val_data[indicator_val > 0], predicted_val_ratings[indicator_val > 0]))
                    inner_rmse.append(val_rmse)

                avg_inner_rmse = np.mean(inner_rmse)

                if avg_inner_rmse < best_rmse:
                    best_rmse = avg_inner_rmse
                    best_params = (n_factors, learning_rate, regularization)

# Train the final model with the best hyperparameters on the entire training set
n_factors, learning_rate, regularization = best_params

U_final = np.random.normal(scale=1./n_factors, size=(n_users, n_factors))
V_final = np.random.normal(scale=1./n_factors, size=(n_items, n_factors))

indicator_train_final = (train_data > 0).astype(int)

for epoch in range(20):
    for i in range(n_users):
        for j in range(n_items):
            if indicator_train_final[i, j] > 0:
                prediction = np.dot(U_final[i, :], V_final[j, :])
                error = train_data[i, j] - prediction

                U_final[i, :] += learning_rate * (error * V_final[j, :] - regularization * U_final[i, :])
                V_final[j, :] += learning_rate * (error * U_final[i, :] - regularization * V_final[i, :])

# Predict the test ratings
predicted_test_ratings = np.dot(U_final, V_final.T)

# Calculate RMSE for test set
indicator_test = (test_data > 0).astype(int)
test_rmse = np.sqrt(mean_squared_error(test_data[indicator_test > 0], predicted_test_ratings[indicator_test > 0]))

print(f'Best Hyperparameters: n_factors={n_factors}, learning_rate={learning_rate}, regularization={regularization}')
print(f'Test RMSE: {test_rmse}')

IndexError: index 774 is out of bounds for axis 0 with size 774

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Load the dataset
# Assuming df is your dataframe with columns: UserID, MovieID, Rating
n_users = df['UserID'].nunique()
n_items = df['MovieID'].nunique()

# Create a user-item matrix
user_item_matrix = df.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0).to_numpy()

# Hyperparameters to optimize
n_factors_list = [10, 20, 50]
learning_rate_list = [0.001, 0.01, 0.1]
regularization_list = [0.01, 0.1, 1]

# K-Fold Cross Validation
outer_kf = KFold(n_splits=5, shuffle=True, random_state=42)
inner_kf = KFold(n_splits=3, shuffle=True, random_state=42)

best_params = None
best_rmse = float('inf')

for train_index, test_index in outer_kf.split(user_item_matrix):
    train_data = user_item_matrix[train_index]
    test_data = user_item_matrix[test_index]

    for n_factors in n_factors_list:
        for learning_rate in learning_rate_list:
            for regularization in regularization_list:
                inner_rmse = []

                for inner_train_index, inner_val_index in inner_kf.split(train_data):
                    inner_train_data = train_data[inner_train_index]
                    inner_val_data = train_data[inner_val_index]

                    # Initialize user and item matrices
                    U = np.random.normal(scale=1./n_factors, size=(inner_train_data.shape[0], n_factors))
                    V = np.random.normal(scale=1./n_factors, size=(inner_train_data.shape[1], n_factors))

                    # Indicator matrix for existing ratings
                    indicator_train = (inner_train_data > 0).astype(int)
                    indicator_val = (inner_val_data > 0).astype(int)

                    # Training with SGD
                    for epoch in range(20):
                        for i in range(inner_train_data.shape[0]):
                            for j in range(inner_train_data.shape[1]):
                                if indicator_train[i, j] > 0:
                                    # Compute the prediction error
                                    prediction = np.dot(U[i, :], V[j, :])
                                    error = inner_train_data[i, j] - prediction

                                    # Update user and item latent factors
                                    U[i, :] += learning_rate * (error * V[j, :] - regularization * U[i, :])
                                    V[j, :] += learning_rate * (error * U[i, :] - regularization * V[j, :])

                    # Predict the validation ratings
                    predicted_val_ratings = np.dot(U, V.T)

                    # Calculate RMSE for validation set
                    val_rmse = np.sqrt(mean_squared_error(inner_val_data[indicator_val > 0], predicted_val_ratings[indicator_val > 0]))
                    inner_rmse.append(val_rmse)

                avg_inner_rmse = np.mean(inner_rmse)

                if avg_inner_rmse < best_rmse:
                    best_rmse = avg_inner_rmse
                    best_params = (n_factors, learning_rate, regularization)

# Train the final model with the best hyperparameters on the entire training set
n_factors, learning_rate, regularization = best_params

U_final = np.random.normal(scale=1./n_factors, size=(train_data.shape[0], n_factors))
V_final = np.random.normal(scale=1./n_factors, size=(train_data.shape[1], n_factors))

indicator_train_final = (train_data > 0).astype(int)

for epoch in range(20):
    for i in range(train_data.shape[0]):
        for j in range(train_data.shape[1]):
            if indicator_train_final[i, j] > 0:
                prediction = np.dot(U_final[i, :], V_final[j, :])
                error = train_data[i, j] - prediction

                U_final[i, :] += learning_rate * (error * V_final[j, :] - regularization * U_final[i, :])
                V_final[j, :] += learning_rate * (error * U_final[i, :] - regularization * V_final[j, :])

# Predict the test ratings
predicted_test_ratings = np.dot(U_final, V_final.T)

# Calculate RMSE for test set
indicator_test = (test_data > 0).astype(int)
test_rmse = np.sqrt(mean_squared_error(test_data[indicator_test > 0], predicted_test_ratings[indicator_test > 0]))

print(f'Best Hyperparameters: n_factors={n_factors}, learning_rate={learning_rate}, regularization={regularization}')
print(f'Test RMSE: {test_rmse}')

IndexError: boolean index did not match indexed array along dimension 0; dimension is 774 but corresponding boolean dimension is 387

Data Preparation: The user-item matrix is created from the dataframe.
Hyperparameter Lists: Lists of hyperparameters to be optimized.
K-Fold Cross Validation: Outer and inner k-fold cross-validation splits.
Training with SGD: The user and item matrices are updated iteratively to minimize the loss function, which includes a regularization term to prevent overfitting.
Hyperparameter Optimization: The best hyperparameters are selected based on the average RMSE from the inner folds.
Final Model Training: The final model is trained on the entire training set using the best hyperparameters.
Evaluation: The RMSE is calculated on the test set to evaluate the model's performance.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import joblib
from xgboost import XGBClassifier

def evaluate_model(model, X, y, dataset_name=""):
    predictions = model.predict(X)
    acc = accuracy_score(y, predictions)
    prec = precision_score(y, predictions, average='weighted')
    rec = recall_score(y, predictions, average='weighted')
    f1 = f1_score(y, predictions, average='weighted')

    print(f"{dataset_name} Set Performance:")
    print(f"Accuracy: {acc:.2f}")
    print(f"Precision: {prec:.2f}")
    print(f"Recall: {rec:.2f}")
    print(f"F1 Score: {f1:.2f}")

    return acc, prec, rec, f1

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=XGBClassifier(tree_method='hist', device='cuda', eval_metric='merror'),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring=f1_weighted_scorer,
    n_jobs=-1
)

# Define output lists
outer_scores = []
val_scores = []
train_scores = []
best_params_list = []
all_param_scores = []

# Perform nested CV on the training data
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the training set
    train_scores.append(evaluate_model(best_model, X_train_fold, y_train_fold, "Training"))

    # Evaluate on the validation set
    val_scores.append(evaluate_model(best_model, X_val_fold, y_val_fold, "Validation"))

    # Store the outer fold score (f1_weighted)
    outer_scores.append(random_search.best_score_)

    # Store all parameter configurations and their scores
    all_param_scores.append(random_search.cv_results_['mean_test_score'])

# Print the performance of the nested CV
print(f"Nested Cross-Validation F1 Score: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print training set performance for each fold
train_scores = np.array(train_scores)
print("Training Set Performance (Averaged):")
print(f"Accuracy: {train_scores[:, 0].mean():.2f} ± {train_scores[:, 0].std():.2f}")
print(f"Precision: {train_scores[:, 1].mean():.2f} ± {train_scores[:, 1].std():.2f}")
print(f"Recall: {train_scores[:, 2].mean():.2f} ± {train_scores[:, 2].std():.2f}")
print(f"F1 Score: {train_scores[:, 3].mean():.2f} ± {train_scores[:, 3].std():.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance (Averaged):")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Print F1 scores of different parameter configurations tried during hyperparameter tuning
print("F1 scores of different parameter configurations tried during hyperparameter tuning:")
for i, param_scores in enumerate(all_param_scores):
    print(f"Fold {i+1}: {param_scores}")

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train best model on the entire training data
best_model = XGBClassifier(**best_params, tree_method='hist', device='cuda', eval_metric='merror', random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on the training set
evaluate_model(best_model, X_train, y_train, "Training")

# Evaluate on the test set
evaluate_model(best_model, X_test, y_test, "Test")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

joblib.dump(best_model, "/content/drive/My Drive/Thesis/Models/Baseline_XGB_training_info.pkl")

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:12:28] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Nested Cross-Validation F1 Score: 0.41 ± 0.00
Validation Set Performance (All Configurations):
Accuracy: 0.44 ± 0.00
F1 Score: 0.42 ± 0.01
Validation Set Performance (Best Model):
Accuracy: 0.44 ± 0.00
F1 Score: 0.42 ± 0.01
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}
Fold 2: {'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.2}
Fold 3: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.1}
Fold 4: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}
Fold 5: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.2}
Training set performance for best performing model:
Accuracy: 0.61
F1 Score: 0.60
Test set performance for best performing model:
Accuracy: 0.44
F1 Score: 0.43
Parameters for the best performing model:
{'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Thesis/Models/Baseline_XGB_25/11.pkl'

## **XGBoost baseline zonder training set**

In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import KFold, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

In [ ]:
# Load in the data
drive.mount('/content/drive')
df_final = pd.read_csv('/content/drive/My Drive/Thesis/Data/df_final_2.csv')

df_final = df_final.astype(np.int32)

Mounted at /content/drive


In [ ]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the "Rating" column
df_final['Rating'] = label_encoder.fit_transform(df_final['Rating'])

In [ ]:
# Define feature and target
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

# Define the parameter grid for tuning
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
}

# Define the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric
f1_weighted_scorer = make_scorer(f1_score, average='weighted')

In [ ]:
# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=XGBClassifier(tree_method='hist', device='cuda',
                            eval_metric='merror'),
    param_distributions=param_dist,
    n_iter=15,
    cv=inner_cv,
    scoring=f1_weighted_scorer,
    n_jobs=-1
)

# Define output lists
outer_scores = []
val_scores = []
best_params_list = []

# Perform nested CV on the training data
for train_idx, val_idx in outer_cv.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold = X_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]

    # Fit the model on the training fold
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_

    # Save the best parameters for this fold
    best_params_list.append(random_search.best_params_)

    # Evaluate on the validation set
    val_predictions = best_model.predict(X_val_fold)
    val_acc = accuracy_score(y_val_fold, val_predictions)
    val_prec = precision_score(y_val_fold, val_predictions, average='weighted')
    val_rec = recall_score(y_val_fold, val_predictions, average='weighted')
    val_f1 = f1_score(y_val_fold, val_predictions, average='weighted')

    # Store validation metrics
    val_scores.append((val_acc, val_prec, val_rec, val_f1))

    # Store the outer fold score (f1_weighted)
    outer_scores.append(random_search.best_score_)

# Print the performance of the nested CV
print(f"Nested Cross-Validation F1 Score: {np.mean(outer_scores):.2f} ± {np.std(outer_scores):.2f}")

# Print validation set performance for each fold
val_scores = np.array(val_scores)
print("Validation Set Performance:")
print(f"Accuracy: {val_scores[:, 0].mean():.2f} ± {val_scores[:, 0].std():.2f}")
print(f"Precision: {val_scores[:, 1].mean():.2f} ± {val_scores[:, 1].std():.2f}")
print(f"Recall: {val_scores[:, 2].mean():.2f} ± {val_scores[:, 2].std():.2f}")
print(f"F1 Score: {val_scores[:, 3].mean():.2f} ± {val_scores[:, 3].std():.2f}")

# Print the best parameters found during hyperparameter tuning
print("Best parameters found during hyperparameter tuning:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")

# Define the best parameters found in nested CV
best_params = best_params_list[np.argmax(outer_scores)]

# Train best model on the entire training data
best_model = XGBClassifier(**best_params, tree_method='hist',
                           device='cuda', eval_metric='merror',
                           random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on the test set
test_predictions = best_model.predict(X_test)
print("Test set performance for best performing model:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.2f}")
print(f"Precision: {precision_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, test_predictions, average='weighted'):.2f}")
print(f"F1 Score: {f1_score(y_test, test_predictions, average='weighted'):.2f}")

# Print the parameters for the best-performing model
print("Parameters for the best performing model:")
print(best_params)

# Save the best performing model
joblib_file = "/content/drive/My Drive/Thesis/Models/Baseline_XGB.pkl"
joblib.dump(best_model, joblib_file)


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:46:48] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Nested Cross-Validation F1 Score: 0.41 ± 0.00
Validation Set Performance:
Accuracy: 0.44 ± 0.00
Precision: 0.43 ± 0.00
Recall: 0.44 ± 0.00
F1 Score: 0.42 ± 0.00
Best parameters found during hyperparameter tuning:
Fold 1: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}
Fold 2: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.1}
Fold 3: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.2}
Fold 4: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.1}
Fold 5: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}
Test set performance for best performing model:
Accuracy: 0.44
Precision: 0.44
Recall: 0.44
F1 Score: 0.43
Parameters for the best performing model:
{'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.2}


['/content/drive/My Drive/Thesis/Models/Baseline_XGB.pkl']